# L5 Lidar and Radar Fusion with Kalman Filters in C++

* Combining data from multiple sensors (as opposed to combining sequential data from one sensor) using Kalman Filters
* Estimate heading, location and speed of pedestrian in front of our car.

![](images/5-2.png)
### Lesson map

* Build an Extended Kalman Filter: Can handle more complex motion and measurement models

Processing flow:
![](images/5-1.png)

1. Lidar and Radar: used to measure state and velocity of a pedestrian
2. Estimation triggered:
    - Predict: Predict pedestrian state (taking into account lapse time)
    - Apply filter depending on typo of measurement received:
        - If measurement provided by laser, apply Kalman Filter.
        - If measurement provided by Radar (non-linear (polar) measurement), may apply Extended Kalman Filter.

![](images/5-3.png)

### Flow
* Each sensor has its own prediction and update scheme. With multiple sensors, beliefs are updated asynchronously.

![](images/5-4.png)
Notes:
* Laser measurement received at time k+1, radar measurement received at time k+2.
* If laser and radar measurements arrive at the same time, predict and update for one sensor first (e.g. laser) and then predict and update for the next sensor. The order doesn't matter.

### Kalman Filter Equations (1D case)


![](images/5-5.png)
![](images/5-6.png)
![](images/5-7.png)
Notes:
* In the linear model, (1) velocity is constant and (2) our sensor only senses the pedestrian position.


Note 
* we do not include the noise in our prediction even though it's in the equations because the noise has mean zero.
* But we can see that the noise covariance Q is added here to the state covariance prediction so that the state uncertainty always increases through the process noise.


### 2D case

Need to define 
* state prediction matrix F and 
* process covariance matrix Q
    * to map stochastic part of the state prediction function

![](images/5-9.png)
![](images/5-10.png)
![](images/5-11.png)
* New 4d state vector (position and velocity for each direction x, y)
* Use same linear model with constant velocity
    * If the pedestrian randomly changes her velocity but the mean change is zero, this introduces process noise in the tracking process.
* Delta t is not constant any more.
    * Uncertainty increases as Delta t increases.
* -> Process noise depends on elapsed time AND the uncertainty of acceleration.

![](images/5-12.png)
![](images/5-13.png)
![](images/5-14.png)
RHS contains random acceleration component.

![](images/5-15.png)
G does not contain random variables, so can take out of the expectation.
![](images/5-16.png)
0 because assumed uncorrelated noise.

![](images/5-17.png)
Note Q_v may sometimes be denoted as Q

### Laser measurements

![](images/5-18.png)
Need to define
* Measurement vector z
* Measurement matrix H
    * 4 x 2
* Covariance matrix R
    * 2 x 2

Analysed point cloud to determine 2d position.
![](images/5-19.png)
![](images/5-20.png)
Suppose pedestrian is moving in a figure 8.
Estimated trajectory may be different depending on process noise (actual and estimate of)
![](images/5-21.png)
![](images/5-22.png)

![](images/5-23.png)
Linear model does not work as well when the pedestrian is not moving in a straight line. Our estimate underestimates the truth.
-> Use circular motion model.


#### Program a Kalman Filter that can take lidar measurements and track a pedestrian in 2D
* Different from previous assigment: Only process most recent measurement each time
* raw_measurement contains only x, y position as we are using laser measurements.
* Call predict before update because delta_t is no longer constant.

### Radar Measurements

Radar observes radial velocity directly using the Doppler effect.
![](images/5-24.png)
![](images/5-25.png)
* Same state transition function with same linear motion model and process noise

* X axis: vehicle's direction of motion
![](images/5-26.png)
![](images/5-27.png)
* Radar measurement covariance matrix: assumes measurement noise is not correlated.

* h specifies how predicted position and speed relate to range, bearing and range rate.
    * Note this is a non-linear function, so no matrix H.
* Putting the Gaussian through a non-linear function like h results in a distribution that is not Gaussian.
    * Example: Transform simulated Gaussian using arctan -> not a Gaussian distribution.
    ![](images/5-28.png)
* Extended Kalman Filter (EKF) uses a linear approximation of h(x) (First Order Taylor Expansion).
    * Example: Transform simulated Gaussian using  a linear approximation of arctan -> Gaussian.
![](images/5-29.png)

#### Linearise measurement function h(x)
* Generalise 1-D linearise h to many dimensions -> many partial derivatives, use a Jacobian.

![](images/5-30.png)
![](images/5-31.png)
![](images/5-32.png)

* Know shape of H_j (3 x 4) because measurement z is a 3-dim vector, state x is a 4-dim vector



#### Calculate the Jacobian

### EKF vs Kalman Filter

5-33

State transition matrix F
Measurement matrix H
EKF:
* Linearise non-linear prediction and measurement functions
    * Replace F, H by corresponding Jacobians which have to be re-calculated at every point in time because linearisation points change
    * But here have linear motion model, so only need to compute Jacobian for measurement function h.
* Use same Kalman filter mechanism as KF

Note: if  f and h are linear, then EKF generates the same result as KF.

### Recap: Sensor Fusion General Flow

* Process measurement triggered every time we receive a (laser or radar) measurement
    * First time call initailise
    * Subsequent times: Predict 
...

### Evaluating KF Performance

![](images/5-34.png)
* Most common metric: Root Mean Squared Error (lower is better).
(img)